In [1]:
import torch

In [4]:
_ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])

In [6]:
img_t = torch.randn(3, 5, 5)
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [7]:
batch_t = torch.randn(2, 3, 5, 5)

In [9]:
img_gray_naive = img_t.mean(-3)
img_gray_naive.size()

torch.Size([5, 5])

In [10]:
batch_gray_naive = batch_t.mean(-3)
batch_gray_naive.size()

torch.Size([2, 5, 5])

In [11]:
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [12]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
unsqueezed_weights.size()

torch.Size([3, 1, 1])

In [13]:
img_weights = (img_t * unsqueezed_weights)
img_weights.size()

torch.Size([3, 5, 5])

In [14]:
batch_weights = (batch_t * unsqueezed_weights)
batch_weights.size()

torch.Size([2, 3, 5, 5])

In [15]:
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)

In [16]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
img_gray_weighted_fancy.size()

torch.Size([5, 5])

In [17]:
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.size()

torch.Size([2, 5, 5])

In [19]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [20]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')

In [21]:
img_named

tensor([[[ 0.0081,  0.5150,  1.2660,  2.1733, -0.0683],
         [ 0.6512,  0.5253, -0.3723,  0.5116,  0.0820],
         [-2.0917, -0.3977,  0.5630,  2.5430, -0.6648],
         [-0.5076, -0.6024, -0.9987,  1.9710,  1.2189],
         [ 0.5588, -0.3962,  0.3204, -0.7718,  0.6789]],

        [[ 0.5043, -0.4461,  0.7903,  0.4151, -0.5630],
         [ 0.8580,  0.7048, -0.0706, -0.7413,  0.5129],
         [-2.1539,  1.4260, -2.3368, -0.6770, -1.1215],
         [-0.0143, -1.0816,  0.4368,  0.3366, -0.3172],
         [ 0.0543, -0.5328, -0.8046, -0.8650,  0.0028]],

        [[-1.0791,  2.2771, -0.0250, -1.0190,  2.4036],
         [-2.6009,  1.8268, -0.5834, -0.2618,  0.4581],
         [ 0.5052,  0.7184,  0.2182, -1.4999, -0.9711],
         [ 0.9351, -2.1090, -0.0479,  0.1919, -0.6953],
         [ 0.9191,  0.1636, -0.1076,  0.3728,  0.1868]]],
       names=('channels', 'rows', 'columns'))

In [23]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned

tensor([[[0.2126]],

        [[0.7152]],

        [[0.0722]]], names=('channels', 'rows', 'columns'))

In [24]:
img_named * weights_aligned

tensor([[[ 0.0017,  0.1095,  0.2692,  0.4620, -0.0145],
         [ 0.1384,  0.1117, -0.0791,  0.1088,  0.0174],
         [-0.4447, -0.0846,  0.1197,  0.5406, -0.1413],
         [-0.1079, -0.1281, -0.2123,  0.4190,  0.2591],
         [ 0.1188, -0.0842,  0.0681, -0.1641,  0.1443]],

        [[ 0.3607, -0.3191,  0.5652,  0.2969, -0.4026],
         [ 0.6136,  0.5041, -0.0505, -0.5302,  0.3669],
         [-1.5404,  1.0199, -1.6713, -0.4842, -0.8021],
         [-0.0102, -0.7736,  0.3124,  0.2407, -0.2269],
         [ 0.0389, -0.3811, -0.5755, -0.6186,  0.0020]],

        [[-0.0779,  0.1644, -0.0018, -0.0736,  0.1735],
         [-0.1878,  0.1319, -0.0421, -0.0189,  0.0331],
         [ 0.0365,  0.0519,  0.0158, -0.1083, -0.0701],
         [ 0.0675, -0.1523, -0.0035,  0.0139, -0.0502],
         [ 0.0664,  0.0118, -0.0078,  0.0269,  0.0135]]],
       names=('channels', 'rows', 'columns'))

In [25]:
try:
    gray_named = (img_named[..., 3] * weights_named).sum('channels')
except Exception as e:
    print(e)

Error when attempting to broadcast dims ['channels', 'rows'] and dims ['channels']: dim 'rows' and dim 'channels' are at the same position from the right but do not match.


In [26]:
gray_plain = gray_named.rename(None)
gray_plain.size(), gray_plain.names

NameError: name 'gray_named' is not defined